<a href="https://colab.research.google.com/github/rajdeepd/bpb-vector-databases/blob/main/chapter10/truelens_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 TruLens Quickstart

In this quickstart you will create a RAG from scratch and learn how to log it and get feedback on an LLM response.

For evaluation, we will leverage the "hallucination triad" of groundedness, context relevance and answer relevance.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/truera/trulens/blob/main/examples/quickstart/quickstart.ipynb)

In [1]:
!pip install trulens trulens-providers-openai chromadb openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of poetry-plugin-export to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.1 MB/s eta 0:00:

In [2]:
import os
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
len(OPENAI_API_KEY)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

## Get Data

In this case, we'll just initialize some simple text in the notebook.

In [3]:
uw_info = """
The University of Washington, founded in 1861 in Seattle, is a public research university
with over 45,000 students across three campuses in Seattle, Tacoma, and Bothell.
As the flagship institution of the six public universities in Washington state,
UW encompasses over 500 buildings and 20 million square feet of space,
including one of the largest library systems in the world.
"""

wsu_info = """
Washington State University (WSU), established in 1890, stands as a prominent public research university with its main campus situated in Pullman, Washington.
Beyond Pullman, WSU maintains a robust presence across the state through its multiple campuses, including those in Spokane, Tri-Cities, and Vancouver,
making it the second-largest institution of higher education in Washington. This dispersed network allows WSU to serve diverse communities and address regional needs.

WSU is particularly renowned for its exceptional programs in various fields. Its College of Veterinary Medicine is highly regarded, producing leading practitioners
and researchers. The College of Agricultural, Human, and Natural Resource Sciences plays a vital role in addressing agricultural challenges and advancing
sustainable practices. The Voiland College of Engineering and Architecture offers a wide range of engineering disciplines and architectural studies,
contributing to technological innovation and infrastructure development. Additionally, the College of Pharmacy and Pharmaceutical Sciences is known for
its rigorous training and research in pharmaceutical sciences.

WSU's commitment to research is evident through its numerous research centers and institutes, which contribute to advancements in areas such as
agriculture, energy, health, and technology. The university's research endeavors often involve collaborations with industry partners and government
agencies, fostering a culture of innovation and knowledge transfer.
"""

seattle_info = """
Seattle, nestled on the shores of Puget Sound in the Pacific Northwest, is a vibrant urban center defined by its stunning natural surroundings.
The city is embraced by a tapestry of waterways, majestic mountain ranges, and lush evergreen forests, creating a unique blend of urban and natural landscapes.
Within its boundaries, Seattle boasts thousands of acres of parkland, offering residents and visitors ample opportunities for outdoor recreation and respite.

Seattle has emerged as a global hub for the technology industry, attracting talent and innovation from around the world.
The city's metropolitan area serves as the headquarters for tech giants such as Microsoft and Amazon, driving economic growth and shaping the digital landscape.
This concentration of tech companies has fostered a dynamic ecosystem of startups, venture capital, and technological advancement, contributing to Seattle's reputation as a leading tech city.
"""

starbucks_info = """
Starbucks Corporation, a globally recognized American multinational chain of coffeehouses and roastery reserves,
calls Seattle, Washington, its home. As the world's largest coffeehouse chain, Starbucks has become a cultural phenomenon,
widely regarded as a key representative of the United States' second wave of coffee culture. This wave emphasized the
social experience of coffee consumption, transforming coffeehouses into community gathering spaces. Beyond simply
serving coffee, Starbucks has cultivated a brand that embodies a specific lifestyle and experience,
influencing coffee consumption habits worldwide.
"""

newzealand_info = """
New Zealand, an island nation situated in the southwestern Pacific Ocean, is a land of striking contrasts and breathtaking beauty. It encompasses two primary islands,
the North and South Islands, alongside a vast archipelago of more than 700 smaller islands. This geographical
diversity yields a remarkable range of landscapes, from verdant rainforests and towering mountain ranges to
pristine beaches and shimmering lakes.

New Zealand's cultural tapestry is woven from the threads of both its indigenous Māori heritage and the influences
of European settlers, creating a unique and vibrant national identity. Wellington serves as the nation's capital,
while Auckland stands as its most populous urban center.
"""

## Create Vector Store

Create a chromadb vector store in memory.

In [4]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(
    api_key=os.environ.get("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002",
)


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(
    name="Washington", embedding_function=embedding_function
)

Populate the vector store.

In [5]:
#vector_store.add("uw_info", documents=uw_info)
#vector_store.add("wsu_info", documents=wsu_info)
#vector_store.add("seattle_info", documents=seattle_info)
#vector_store.add("starbucks_info", documents=starbucks_info)
#vector_store.add("newzealand_info", documents=newzealand_info)
vector_store.add(
    documents=[
        uw_info,wsu_info,seattle_info,starbucks_info,newzealand_info

    ],
    ids=["uw_info", "wsu_info","seattle_info","starbucks_info","newzealand_info"]
)

## Build RAG from scratch

Build a custom RAG from scratch, and add TruLens custom instrumentation.

In [6]:
from trulens.apps.app import instrument
from trulens.core import TruSession

session = TruSession()
session.reset_database()

🦑 Initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]
Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [7]:
from openai import OpenAI

oai_client = OpenAI()

In [8]:
from openai import OpenAI

oai_client = OpenAI()


class RAG:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(query_texts=query, n_results=4)
        # Flatten the list of lists into a single list
        return [doc for sublist in results["documents"] for doc in sublist]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        if len(context_str) == 0:
            return "Sorry, I couldn't find an answer to your question."

        completion = (
            oai_client.chat.completions.create(
                model="gpt-4o",
                temperature=0,
                messages=[
                    {
                        "role": "user",
                        "content": f"We have provided context information below. \n"
                        f"---------------------\n"
                        f"{context_str}"
                        f"\n---------------------\n"
                        f"First, say hello and that you're happy to help. \n"
                        f"\n---------------------\n"
                        f"Then, given this information, please answer the question: {query}",
                    }
                ],
            )
            .choices[0]
            .message.content
        )
        if completion:
            return completion
        else:
            return "Did not find an answer."

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query=query)
        completion = self.generate_completion(
            query=query, context_str=context_str
        )
        return completion


rag = RAG()

## Set up feedback functions.

Here we'll use groundedness, answer relevance and context relevance to detect hallucination.

In [9]:
import numpy as np
from trulens.core import Feedback
from trulens.core import Select
from trulens.providers.openai import OpenAI

provider = OpenAI(model_engine="gpt-4")

# Define a groundedness feedback function
f_groundedness = (
    Feedback(
        provider.groundedness_measure_with_cot_reasons, name="Groundedness"
    )
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)
# Question/answer relevance between overall question and answer.
f_answer_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
    .on_input()
    .on_output()
)

# Context relevance between question and each context chunk.
f_context_relevance = (
    Feedback(
        provider.context_relevance_with_cot_reasons, name="Context Relevance"
    )
    .on_input()
    .on(Select.RecordCalls.retrieve.rets[:])
    .aggregate(np.mean)  # choose a different aggregation method if you wish
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.retrieve.rets[:] .


## Construct the app
Wrap the custom RAG with TruCustomApp, add list of feedbacks for eval

In [10]:
from trulens.apps.app import TruApp

tru_rag = TruApp(
    rag,
    app_name="RAG",
    app_version="base",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

instrumenting <class '__main__.RAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query


## Run the app
Use `tru_rag` as a context manager for the custom RAG-from-scratch app.

In [11]:
with tru_rag as recording:
    rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the United States?"
    )
    rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the New Zealand?"
    )
    rag.query("Does Washington State have Starbucks on campus?")

/usr/local/lib/python3.11/dist-packages/trulens/feedback/llm_provider.py:235: UserWarning: No supporting evidence provided. Returning score only.
  warnings.warn(


## Check results

We can view results in the leaderboard.

In [12]:
session.get_leaderboard()

,,Answer Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,
RAG,base,0.833333,1.0,1.716593,0.002069


## Use guardrails

In addition to making informed iteration, we can also directly use feedback results as guardrails at inference time. In particular, here we show how to use the context relevance score as a guardrail to filter out irrelevant context before it gets passed to the LLM. This both reduces hallucination and improves efficiency.

To do so, we'll rebuild our RAG using the @context-filter decorator on the method we want to filter, and pass in the feedback function and threshold to use for guardrailing.

In [13]:
from trulens.core.guardrails.base import context_filter

# note: feedback function used for guardrail must only return a score, not also reasons
f_context_relevance_score = Feedback(
    provider.context_relevance, name="Context Relevance"
)


class FilteredRAG(RAG):
    @instrument
    @context_filter(
        feedback=f_context_relevance_score,
        threshold=0.75,
        keyword_for_prompt="query",
    )
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(query_texts=query, n_results=4)
        if "documents" in results and results["documents"]:
            return [doc for sublist in results["documents"] for doc in sublist]
        else:
            return []


filtered_rag = FilteredRAG()

## Record and operate as normal

In [14]:
from trulens.apps.custom import TruCustomApp

filtered_tru_rag = TruCustomApp(
    filtered_rag,
    app_name="RAG",
    app_version="filtered",
    feedbacks=[f_groundedness, f_answer_relevance, f_context_relevance],
)

with filtered_tru_rag as recording:
    filtered_rag.query(
        query="What wave of coffee culture is Starbucks seen to represent in the United States?"
    )
    filtered_rag.query(
        "What wave of coffee culture is Starbucks seen to represent in the New Zealand?"
    )
    filtered_rag.query("Does Washington State have Starbucks on campus?")

<ipython-input-14-dad4eb8b08ad>:1: DeprecationWarning: from trulens.apps.custom import instrument
        is being deprecated in the next major version; use from trulens.apps.app import instrument
        instead.
  from trulens.apps.custom import TruCustomApp
<ipython-input-14-dad4eb8b08ad>:3: DeprecationWarning: TruCustomApp is being deprecated in the next major version; use TruApp instead.
  filtered_tru_rag = TruCustomApp(


instrumenting <class '__main__.FilteredRAG'> for base <class '__main__.FilteredRAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
	instrumenting retrieve
instrumenting <class '__main__.FilteredRAG'> for base <class '__main__.RAG'>
	instrumenting retrieve
	instrumenting generate_completion
	instrumenting query
	instrumenting retrieve


/usr/local/lib/python3.11/dist-packages/trulens/feedback/llm_provider.py:235: UserWarning: No supporting evidence provided. Returning score only.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/trulens/core/feedback/feedback.py:1023: UserWarning: Feedback function Context Relevance with aggregation <function mean at 0x7b05f4172ca0> had no inputs.
  warnings.warn(


In [16]:
session.get_leaderboard()

Answer Relevance  Context Relevance  Groundedness  \
app_name app_version                                                      
RAG      base                 0.777778           0.083333           1.0   
         filtered             0.333333           1.000000           1.0   

                        latency  total_cost  
app_name app_version                         
RAG      base          1.716593    0.002069  
         filtered     13.491991    0.139217

In [15]:
from trulens.dashboard import run_dashboard

run_dashboard(session)

Starting dashboard ...
npm warn exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://mean-shirts-grow.loca.lt

  Submit this IP Address: 35.243.175.8



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>